# When to Buy a Company Using DCF Calculation

In [81]:
# Imports needed
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
class DCF_Calculator:
    
    def __init__(self,company,api_key,rror):
        """
        Takes in the companies ticker symbol as a string, your API KEY as a string,
        and your required rate of return you want to recieve for your investment
        as an int(ex: an 8.4% return will be passed in as 0.084)
        
        """
        self.company = company 
        self.api_key = api_key
        self.rror = rror
        
    def income_statement(self):
        """
        Gets the last 10years of the revenue, interest expense, income before tax, 
        income tax expense, and net income from the companies income statements and 
        writes it out to a pandas dataframe
        
        """
         # Request to API & Dict that stores the numbers from each year with the date as the key of the dict
        income_state = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{self.company}?apikey={self.api_key}').json()
        revenue = {}
        interest_expense = {}
        income_before_tax = {}
        income_tax_expense = {}
        net_income = {}
        count = 0
        #loops through the JSON data of the income state and get the last 10 Income Statements 
        for item in income_state:
            if count < 10:
                revenue[item['date']] = item['revenue']
                interest_expense[item['date']] = item['interestExpense']
                income_before_tax[item['date']] = item['incomeBeforeTax']
                income_tax_expense[item['date']] = item['incomeTaxExpense']
                net_income[item['date']]= item['netIncome']
                count += 1
        # Store it all in one dictionary and writes is out to a Pandas Data frame
        income_statement = {}
        income_statement['revenue'],income_statement['interest_expense'],income_statement['income_before_tax'],income_statement['income_tax_expense'],income_statement['net_income'] = revenue, interest_expense,income_before_tax, income_tax_expense, net_income 
        is_df = pd.DataFrame.from_dict(income_statement,orient='index').T
        is_df.sort_index(inplace= True) 
        return is_df
    
    def cash_flow_statement(self):
        """
        Gets the last 10years of the cash flow from operations, capital expenditures,
        and free cash flow from the companies cash flow statements and writes it out 
        to a pandas dataframe
        """
        
        # Request to API & Dict that stores the numbers from each year with the date as the key of the dict
        cash_flow = requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{self.company}?apikey={self.api_key}').json()

        cf_from_operating_activities = {}
        capital_expenditures = {}
        free_cash_flow = {}
        count = 0
        #loops through the JSON data of the income state and get the last 10 Cash Flow statements 
        for item in cash_flow:
            if count < 10:
                cf_from_operating_activities[item['date']] = item['operatingCashFlow']
                capital_expenditures[item['date']] = item['capitalExpenditure']
                free_cash_flow[item['date']] = item['freeCashFlow']
                count += 1
        # Store it all in one dictionary and writes is out to a Pandas Data frame
        cash_flow_state = {}
        cash_flow_state["cfo"],cash_flow_state["capex"],cash_flow_state["free_cash_flow"] = cf_from_operating_activities, capital_expenditures,free_cash_flow 
        cf_df = pd.DataFrame.from_dict(cash_flow_state,orient='index').T
        cf_df.sort_index(inplace = True)
        return cf_df
    
    def balance_sheet_statement(self):
        """
        Gets the last 10years of the short term debt, long term debt, and the cash on hand plus
        short term investments from the companies balance sheet statements and writes it out 
        to a pandas dataframe
        
        """
        # Request to API & dict that stores the numbers from each year with the date as the key of the dict
        balance_sheet  = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{self.company}?apikey={self.api_key}').json()

        short_term_debt = {}
        long_term_debt = {}
        cash_and_short_term_invest = {}
        count = 0
        #loops through the JSON data of the income state and get the last 10 Balance sheet statements 
        for item in balance_sheet:
            if count < 10:
                short_term_debt[item['date']] = item['totalCurrentLiabilities']
                long_term_debt[item['date']] = item['longTermDebt']
                cash_and_short_term_invest[item['date']] = item['cashAndShortTermInvestments']
                count += 1
        # Store it all in one dictionary and writes is out to a Pandas Data frame
        balance_sheet_state = {}
        balance_sheet_state["short_term_debt"],balance_sheet_state["long_term_debt"],balance_sheet_state["cash_and_sti"] = short_term_debt,long_term_debt,cash_and_short_term_invest
        bs_df = pd.DataFrame.from_dict(balance_sheet_state,orient='index').T
        bs_df.sort_index(inplace= True)

        return bs_df
    
    def fcf_to_net_income_margins(self):
        """ Calculates the average free cash flow to net income margins over the 10 year 
        period by dividing the free cash flow by net income for each year and returns the %.
        
        """
        fcf_to_net_income_margin = (self.cash_flow_statement().free_cash_flow/self.income_statement().net_income).mean()
        return fcf_to_net_income_margin
    
    def revenue_growth_rate(self):
        """
        Calculates the average revenue growth rate over a 10 year period and returns it the %.
        (rev2020 - rev2019)/rev(2020)
        """
        # Revenue growth rate average over a 10 year period 
        revenue_growth_rate = self.income_statement().revenue.pct_change().mean()
        
        return revenue_growth_rate
    
    def revenue_projections(self):
        """
        Projects out 4 years of the revenue based on the average growth rate, starting from 
        the most recent revenue reporting. last_rev(1+rev_growth_rate) and returns a dataframe  
        """
        growth_rate = self.revenue_growth_rate()
        # Revenue Projection calculation with write out to a dataframe 
        recent_year_data = 2019
        project_year = [recent_year_data+i for i in range(5)]
        rev_projection = {}
        rev_projection[project_year[1]] = self.income_statement().revenue.iloc[-1] * (1+growth_rate)
        rev_projection[project_year[2]] = rev_projection[project_year[1]] * (1+growth_rate)
        rev_projection[project_year[3]] = rev_projection[project_year[2]] * (1+growth_rate)
        rev_projection[project_year[4]] = rev_projection[project_year[3]] * (1+growth_rate)
        rev_projection_df = pd.Series(rev_projection)
        return rev_projection_df
    def revenue_to_net_income_margins(self):
        """ Calculates the average revenue to net income margins over the 10 year 
        period by dividing the net income by revenue for each year and returns the %."""
        
        revenue_to_net_income_margin = (self.income_statement().net_income/self.income_statement().revenue).mean()
        return revenue_to_net_income_margin
    def net_income_projections(self):
        """
        Projects out 4 years of the net income based on the average rev to net income margins, 
        using our revenue projections. (Our revenue projections*revenue to net income margins) 
        and returns a dataframe
        """
        # Net income projections using revenue to net income margins
        net_income_projection = self.revenue_projections() * self.revenue_to_net_income_margins()
        net_income_projection_df = pd.Series(net_income_projection)
        
        return net_income_projection
    
    
    def free_cash_flow_projections(self):
        """
        Now we project out out free cash flow using our projected net income and our
        free cash flow to net income margins and returns a dataframe
        """
        
        # Multiples net income by the free cash flow to net income margins 
        free_cash_flow_projections = self.net_income_projections() * self.fcf_to_net_income_margins()
        free_cash_flow_projections_df  = pd.DataFrame(free_cash_flow_projections)
        
        return free_cash_flow_projections_df
    
    def companies_value(self):
        """
        Calculates the price we should buy the company by calculating the temrinal value,
        picking a perpetual growth rate, factor in the shares outstanding and applying our
        discount rate.
        """
        # Company quote: Has the shares outstanding, has marketcap, eps, pe
        company_quote = requests.get(f'https://financialmodelingprep.com/api/v3/quote/{self.company}?apikey={self.api_key}').json()
        company_profile = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{self.company}?apikey={self.api_key}').json()
        
        # Perpetual Growth rate: How long we want the company to grow forever
        perpetual_growth_rate = 0.025
        
        # Shares outstanding 
        shares_outstanding = 0
        for item in company_quote:
            shares_outstanding = item['sharesOutstanding']

        # Calculating companies value

        # Calculating Terminal Value and creates a new df of the free cashflow projection w/o recent number 
        terminal_value  = self.free_cash_flow_projections().iloc[-1] *(1+perpetual_growth_rate) /(self.rror-perpetual_growth_rate)
        company_value_df = pd.DataFrame(self.free_cash_flow_projections())
        
        # Adds the terminal value to the DF 
        company_value_df = company_value_df.append(terminal_value,ignore_index=True)

        # Getting the discounted factor (1+rror)^time_period

        discount_factor = [(1+self.rror)**i for i in range(1,5)]

        # repeats the last time period number for the terminal value and adds it to our Df
        discount_factor.append(discount_factor[-1])
        discount_factor_df = pd.DataFrame(discount_factor)

        # Divides our projected free cashflow value by our discounted factor
        final_fcf = company_value_df.div(discount_factor_df)

        # Total all free cash flow with our discount factored in to get companies value
        total_company_value = final_fcf.values.sum()

        # Calculates the buy price buy taking total value and dividing by shares outstanding
        intrinsic_value = total_company_value/shares_outstanding

        # Gets the total debt of the company with short Investments & cash on hand factored in
        net_borrowings = self.balance_sheet_statement().short_term_debt.iloc[-1] + self.balance_sheet_statement().long_term_debt.iloc[-1] - self.balance_sheet_statement().cash_and_sti.iloc[-1]
        
        # Calaculates the buy price with the companies debt factored in to its's value
        intrinsic_value_w_debt = (total_company_value - net_borrowings)/shares_outstanding

        # Debt Analysis
        # Weight of Debt and Equity. 
        # Total debt = short term debt _ long term debt 
        #Total capital = Total debt + market cap 
        # Weight of debt = total debt / total capital 
        total_debt = self.balance_sheet_statement().short_term_debt.iloc[-1]+ self.balance_sheet_statement().long_term_debt.iloc[-1]
        market_cap = 0
        for item in company_quote:
            market_cap = item['marketCap']

        total_captial = total_debt + market_cap

        weight_of_debt = total_debt / total_captial 

        weight_of_asset = (1-weight_of_debt)

        # Data plots needed 
        self.income_statement().revenue.plot(kind='bar',title = 'Revenue', x = 'Year', y = 'Revenue in Billions')
        self.income_statement().net_income.plot(kind = 'bar', title = 'Net Income')
        self.cash_flow_statement().free_cash_flow.plot(kind = 'bar', title = 'Free Cash Flow')

        revenue_to_net_income_to_fcf_df = pd.DataFrame({'Revenue':self.income_statement().revenue,"Net Income":self.income_statement().net_income,"Free Cash Flow":self.cash_flow_statement().free_cash_flow})
        revenue_to_net_income_to_fcf_df.plot(kind= 'bar',title = 'Rev to Net Income to FCF')
        
        # Writes out all the important data to a CSV file on your local computer.
        output_data = {}

        output_data["Buy Price w debt factored in $"], output_data["Buy Price w/o debt factored in $"], output_data["Revenue Growth Rate %"], output_data["FCF to net income margins %"],output_data["Revenue to net income margins %"], output_data["Weight of Asset %"], output_data["Weight of Debt"]=intrinsic_value_w_debt,intrinsic_value,self.revenue_growth_rate()*100,self.fcf_to_net_income_margins()*100,self.revenue_to_net_income_margins()*100,weight_of_asset*100,weight_of_debt*100

        output_data_df = pd.DataFrame.from_dict(output_data,orient='index')
        output_data_df.to_csv(r'/Users/faithful/Desktop/Google.csv')
        
        print(f"Fair Value with debt factored in: ${intrinsic_value_w_debt}")
        print(f"Fair Value without debt: ${intrinsic_value}")
        print(f"Revenue Growth Rate: {self.revenue_growth_rate()*100}")
        print(f"Free Cash Flow to Net Income Margins: {self.fcf_to_net_income_margins()*100}")
        print(f"Revenue to Net Income Margins: {self.revenue_to_net_income_margins()*100}")
        print(f"Weight of Asset: {weight_of_asset*100}")
        print(f"Weight of Debt: {weight_of_debt*100}")
        print(final_fcf)
        print(company_value_df)
        print(discount_factor_df)
        print(total_company_value)
        print(self.revenue_projections())
        print(self.free_cash_flow_projections())
    
    